In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

        try:
            title = soup.find("span", attrs={"class":'B_NuCI'}).text.strip()

        except AttributeError:
            title = ""

        return title

# Function to extract Product Price
def get_price(soup):

        try:
            price = soup.find("div", attrs={'class':'_30jeq3 _16Jk6d'}).string.strip()

        except AttributeError:
            price = ""

        return price
# Function to extract Actual Product Price
def get_actual_price(soup):

        try:
            price = soup.find("div", attrs={'class':'_3I9_wc _2p6lqe'}).text

        except AttributeError:
            price = ""

        return price
# Function to extract Product Rating
def get_rating(soup):

        try:
            rating = soup.find("div", attrs={'class':'_3LWZlK'}).text

        except AttributeError:
            rating = ""	

        return rating

# Function to extract Discounts
def get_discount(soup):
        try:
            discount = soup.find("div", attrs={'class':'_3Ay6Sb _31Dcoz'})
            discount = discount.find("span").text

        except AttributeError:
            discount = ""	

        return discount

# Function to extract Highlights
def get_highlight(soup):
        try:
            highlight = soup.find("div", attrs={'class':'_2418kt'})
            highlight= highlight.find("li", attrs={'class':'_21Ahn-'}).text

        except AttributeError:
            highlight = ""	

        return highlight

In [3]:
if __name__ == '__main__':

    # The webpage URL
    url = 'https://www.flipkart.com/mobiles/pr?sid=tyy%2C4io&otracker=categorytree'
    
    # HTTP Request
    source = requests.get(url)
    
    # Soup Object containing all data
    soup = BeautifulSoup(source.content, 'html.parser')
              
    # Fetch links as List of Tag Objects
    pages = soup.find_all("a", attrs={'class':'ge-49M'})  

    # Store the links
    page_list =[]
    
    # Dictionary 
    d = {"Title":[], "Price":[], "ActualPrice":[],"Discount":[],"Rating":[],"Highlight":[]}

    # Loop for extracting pages from Tag Objects
    for page in pages:

        page_list.append(page.get('href'))
      
      
    # Loop for extracting product details from each page
    for page in page_list:
        new_page = requests.get("https://www.flipkart.com" + page)

        new_page = BeautifulSoup(new_page.content, "html.parser") 

        new_page = soup.find_all("a", attrs={'class':'_1fQZEK'})
        
        links_list = []
                   
        for link in new_page:
            links_list.append(link.get('href'))

        for link in links_list:
            new_webpage = requests.get("https://www.flipkart.com" + link)

            new_soup = BeautifulSoup(new_webpage.content, "html.parser")

            # Function calls to display all necessary product information
            d['Title'].append(get_title(new_soup))
            d['Price'].append(get_price(new_soup))
            d['ActualPrice'].append(get_actual_price(new_soup))
            d['Discount'].append(get_discount(new_soup))
            d['Rating'].append(get_rating(new_soup))
            d['Highlight'].append(get_highlight(new_soup))



    
    flipkart_df = pd.DataFrame.from_dict(d)
    #flipkart_df['title'].replace('', np.nan, inplace=True)
    #flipkart_df = flipkart_df.dropna(subset=['title'])
    #flipkart_df.to_csv("flipkart_data.csv", header=True, index=False)        
    
          
            

In [5]:
flipkart_df
flipkart_df.to_csv("flipkart_data.csv", header=True, index=False)